In [13]:
##Go to Main page to load all options
import requests
from bs4 import BeautifulSoup
import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


path = 'https://sea.cc.ntpu.edu.tw/pls/dev_stud/course_query_all.CHI_query_common'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

r = requests.get(path, headers = headers)
r.encoding = 'big5'

soup = BeautifulSoup(r.text, 'lxml')
elem = soup.find('select')


schoolList = []
for item in elem.find_all('option'):
    schoolList.append(item.string)
schoolList.pop(0)

print(schoolList)

In [113]:
##use options above to Generate to-be-crawled url 
def GetAllHtmls(SchoolOption):

    postpath = 'https://sea.cc.ntpu.edu.tw/pls/dev_stud/course_query_all.queryByAllConditions'

    FormData = {
    'qCollege': SchoolOption.encode('big5'),
    'qYear':105,
    'qTerm':2,
    'seq1':'A',
    'seq2':'M'
    }

    r = requests.post(postpath,FormData)
    r.encoding = 'big5'
    
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.find('table')
    
    ##Write Htmls to files in my own memory 
    file = open('TaipeiCourses/'+SchoolOption,'w', encoding='utf8')
    file.write(str(table))

for school in schoolList:
    GetAllHtmls(school)
    print(school + 'done!')

法律學院done!
商學院done!
公共事務學院done!
社會科學學院done!
人文學院done!
電機資訊學院done!
通識教育中心done!


In [170]:
import pandas as pd

def WriteHtmlIntoMongo(Path):
    df_courses = pd.read_html(Path,encoding='utf8')[0]

    ##Change first row to column header
    columns = []
    for column in df_courses.xs(0):
        columns.append(column.replace("、",""))
    df_courses.columns = columns
    df_courses = df_courses.drop(df_courses[df_courses['學期']=='學期'].index[[0]])

    ##remember Note Row
    noteSeries=df_courses[pd.isnull(df_courses['學年'])]['NO.']

    ##remove Note Row
    df_courses.drop(df_courses[pd.isnull(df_courses['學年'])].index, inplace=True)

    ##Transfer Note Row to another column 
    df_courses['備註'] = list(noteSeries)

    ##drop No. column(hav been mixed, no useful information)
    df_courses.drop('NO.',1,inplace=True)
    
    ##replace nan(inpandas) with None in python(which can be identified in MongoDB)  
    df_courses = df_courses.where(df_courses.notnull(), None)
    
    ##Deal with '\t' in '上課時間、教室'
    def dropSpace(item):
        if '\t' in item:
            return item.replace("\t",",")
        return item
    df_courses['上課時間教室'] = df_courses['上課時間教室'].apply(dropSpace)

    ##Mongo allow dictionary only. Before transfer to dict, first store it in string type, otherwise, the type in Mongo can be various
    def ToDIct(Row):
        def ToStr(item):
            if(item == '-'):
                return None
            else:
                return str(item)
        return Row.apply(ToStr).to_dict()
    CoursesDictList = df_courses.apply(ToDIct, axis=1).tolist()
    return CoursesDictList


In [171]:
## you can load this list from the website, it is only for easy use
schoolList = ['法律學院', '商學院', '公共事務學院', '社會科學學院', '人文學院', '電機資訊學院', '通識教育中心']

##Build a list containing all html file paths
PathLi = []
for school in schoolList:
    PathLi.append('TaipeiCourses/'+school)

##Insert into MongoDB
from pymongo import MongoClient
conn = MongoClient('localhost', 27017)
db = conn.SuperUniversityCourses
collection = db['ntpu']

##Use function defined above
for path in PathLi:
    collection.insert_many(WriteHtmlIntoMongo(path))
    
conn.close()

In [157]:
import pandas as pd
df_courses = pd.read_html(PahtLi[0],encoding='utf8')[0]

In [167]:
df_courses

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,NO.,學年,學期,課程流水號,開課系所,應修系級,必選修別,科目名稱,授課教師,類別,學分,繳費時數,授課語別,上課時間、教室,是否開放加簽,加簽人數上限,限修總計人數,已選總計人數,已核准人數,待分發人數
1,1.,105,2,M6002,法碩一般生組,法碩一般生組2,必,論文Thesis (Projects),******,半,0,0,中文,每週未維護0,否,-,200,109,109,0
2,[備註],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.,105,2,M6004,法碩法專組,法碩法專組3,必,論文Thesis (Projects),******,半,0,0,中文,每週未維護0,否,-,40,9,9,0
4,[備註],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.,105,2,M6106,法博,法博2,必,論文Thesis (Projects),******,半,0,0,中文,每週未維護0,否,-,35,22,22,0
6,[備註],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.,105,2,U4008,法律學系法學組,法律學系財經法組3 法律學系財經法組4 法律學系法學組3 法律學系法學組4 法律學系司法組3...,必 必 必 必 必 必,法律實習與社會服務Legal Clinic and Pro Bono Service,杜怡靜,半,0,2,中文,每週日1~2,否,-,121,108,107,0
8,[備註] 本課程限法律學系學士班學生修習。需於指定機構單位完成24小時校外服務並經考核合格。...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5.,105,2,U1588,語言,中文系1 歷史系1 休運系1 語文通識1 法律學系財經法組1 法律學系法學組1 法律學系司法組1,必 必 必 必 必 必 必,大學英文：英語聽講練習 College English: Listening and Spe...,殷雅玲,全,1,2,中文,每週一3~4\t文3F09_L,否,-,40,30,30,0


In [152]:
def dropSpace(item):
    if '\t' in item:
        return item.replace("\t",",")
    return item
df_courses['上課時間、教室'] = df_courses['上課時間、教室'].apply(dropSpace)
df_courses

,學年,學期,課程流水號,開課系所,應修系級,必選修別,科目名稱,授課教師,類別,學分,繳費時數,授課語別,上課時間、教室,是否開放加簽,加簽人數上限,限修總計人數,已選總計人數,已核准人數,待分發人數,備註
1,105,2,M6002,法碩一般生組,法碩一般生組2,必,論文Thesis (Projects),******,半,0,0,中文,每週未維護0,否,-,200,109,109,0,[備註]
3,105,2,M6004,法碩法專組,法碩法專組3,必,論文Thesis (Projects),******,半,0,0,中文,每週未維護0,否,-,40,9,9,0,[備註]
5,105,2,M6106,法博,法博2,必,論文Thesis (Projects),******,半,0,0,中文,每週未維護0,否,-,35,22,22,0,[備註]
7,105,2,U4008,法律學系法學組,法律學系財經法組3 法律學系財經法組4 法律學系法學組3 法律學系法學組4 法律學系司法組3...,必 必 必 必 必 必,法律實習與社會服務Legal Clinic and Pro Bono Service,杜怡靜,半,0,2,中文,每週日1~2,否,-,121,108,107,0,[備註] 本課程限法律學系學士班學生修習。需於指定機構單位完成24小時校外服務並經考核合格。...
9,105,2,U1588,語言,中文系1 歷史系1 休運系1 語文通識1 法律學系財經法組1 法律學系法學組1 法律學系司法組1,必 必 必 必 必 必 必,大學英文：英語聽講練習 College English: Listening and Spe...,殷雅玲,全,1,2,中文,"每週一3~4,文3F09_L",否,-,40,30,30,0,[備註] 中文、歷史、運管、法學、司法、財經法(語言教室)
11,105,2,U1589,語言,中文系1 歷史系1 休運系1 語文通識1 法律學系財經法組1 法律學系法學組1 法律學系司法組1,必 必 必 必 必 必 必,大學英文：英語聽講練習 College English: Listening and Spe...,單傳國,全,1,2,中文,"每週一3~4,文3F01_L",否,-,40,33,33,0,[備註] 中文、歷史、運管、法學、司法、財經法(語言教室)
13,105,2,U1590,語言,中文系1 歷史系1 休運系1 語文通識1 法律學系財經法組1 法律學系法學組1 法律學系司法組1,必 必 必 必 必 必 必,大學英文：英語聽講練習 College English: Listening and Spe...,楊冀光,全,1,2,中文,"每週一1~2,文3F09_L",否,-,40,26,26,0,[備註] 中文、歷史、運管、法學、司法、財經法(語言教室)
15,105,2,U1591,語言,中文系1 歷史系1 休運系1 語文通識1 法律學系財經法組1 法律學系法學組1 法律學系司法組1,必 必 必 必 必 必 必,大學英文：英語聽講練習 College English: Listening and Spe...,彭怡婷,全,1,2,中文,"每週一1~2,文3F10_L",否,-,40,31,31,0,[備註] 中文、歷史、運管、法學、司法、財經法(語言教室)
17,105,2,M5036,法博,法博1 法博2,必 必,刑事訴訟法專題綜合研究Seminar: Omnibus Criminal Procedure...,李榮耕,半,2,2,中文,"每週二5~6,法2F09",否,-,5,1,1,0,[備註] 105.12.22依課程資訊異動申請，新開課程。 1.博士班刑法領域二選一必修課，...
19,105,2,M5039,法博,法博1 法博2,必 必,刑法專題綜合研究Seminar: Omnibus Criminal Law,曾淑瑜,半,2,2,中文,"每週四1~2,法5F16",否,-,5,3,3,0,[備註] 博士班刑法領域二選一必修課，98學年以後入學者必修。


In [ ]:
##Mongo allow dictionary only. Before transfer to dict, first store it in string type, otherwise, the type in Mongo can be various
def ToDIct(Row):
    def ToStrAndRemoveSpace(item):
        if(item == '-'):
            return None
        elif(item != None):
            return str(item)
        else:
            return None
    return Row.apply(ToStrAndRemoveSpace).to_dict()
CoursesDictList = df_courses.apply(ToDIct, axis=1).tolist()
return CoursesDictList

In [151]:
##For Test
schoolList = ['法律學院', '商學院', '公共事務學院', '社會科學學院', '人文學院', '電機資訊學院', '通識教育中心']

PathLi = []
for school in schoolList:
    PathLi.append('TaipeiCourses/'+school)


import pandas as pd
df_courses = pd.read_html(PahtLi[0],encoding='utf8')[0]

##Change first row to column header
columns = []
for column in df_courses.xs(0):
    columns.append(column)
df_courses.columns = columns
df_courses = df_courses.drop(df_courses[df_courses['學期']=='學期'].index[[0]])

##remember Note Row
noteSeries=df_courses[pd.isnull(df_courses['學年'])]['NO.']

##remove Note Row
df_courses.drop(df_courses[pd.isnull(df_courses['學年'])].index, inplace=True)

##Transfer Note Row to another column 
df_courses['備註'] = list(noteSeries)

##drop No. column(hav been mixed, no useful information)
df_courses.drop('NO.',1,inplace=True)

##Deal with '\t' in '上課時間、教室'
def dropSpace(item):
    if '\t' in item:
        return item.replace("\t",",")
    return item
df_courses['上課時間、教室'] = df_courses['上課時間、教室'].apply(dropSpace)

##Mongo allow dictionary only. Before transfer to dict, first store it in string type, otherwise, the type in Mongo can be various
def ToDIct(Row):
    def ToStrAndRemoveSpace(item):
        if(item == '-'):
            return None
        else:
            return str(item)
    return Row.apply(ToStr).to_dict()
CoursesDictList = df_courses.apply(ToDIct, axis=1).tolist()
